In [1]:
from __future__ import annotations
import math
import numpy as np

In [56]:
class Value:
    def __init__(self, data: float | np.ndarray, _children=(), _op=''):
        self.data = data 
        self.grad = 0.0
        self._prev = set(_children)
        self._backward = lambda: None

    def __repr__(self):
        return f'Value(data={self.data}, grad={self.grad})'

    def __getitem__(self, idx):
        if idx<0 or idx>len(self.data):
            raise IndexError(f"Row index {idx} out of range; must be in [0, {len(self.val)-1}]")

        return self.data[idx]

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data + out.grad
        out._backward = _backward
        return out

    def tanh(self):
        y = math.tanh(self.data)
        out = Value(y, (self,), 'tanh')

        def _backward():
            self.grad += (1 - y*y) + out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)); 'Only accepting int, float'
        out = Value(self.data ** other, (self, ), '**') 
        
        def _backward():
            self.grad += other * (self.data) ** (other - 1) * out.grad

        out._backward = _backward
        return out

    def __matmul__(self, other):
        n = len(self.data[0])   # row
        m = len(self.data)      # col   
        p = len(other.data[0])  # row
        q = len(other.data)     # col

        # check conditions -  if internal dimensons dont match -- matmul is not possible
        if m != p:
            raise ValueError(f'Cannot multiply matrices of unequal internal dimensions')

        res = [[0 for _ in range(q)] for _ in range(q)]

        for i in range(n):
            for j in range(q):
                for k in range(m):
                    res[i][j] += self.data[i][k] * other.data[k][j]

        return Value(res) 
        
    def __neg__(self):
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def backward(self):
        visited = set()
        topo = []

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = 1.0
        for x in reversed(topo):
            x._backward()
                
    

# Testing Value class

In [34]:
a = Value(2)
c = Value(3)
b = Value(4)

d = a * b
f = d + c
f

Value(data=11, grad=0.0)

In [35]:
f.grad = 1

In [36]:
f._backward()
d

Value(data=8, grad=1.0)

In [37]:
d._backward()
d

Value(data=8, grad=1.0)

In [38]:
c._backward()

In [39]:
d._backward()

In [40]:
a._backward()
b._backward()

In [41]:
a,b,c,d,f # checking populations of grads

(Value(data=2, grad=8.0),
 Value(data=4, grad=4.0),
 Value(data=3, grad=1.0),
 Value(data=8, grad=1.0),
 Value(data=11, grad=1))

In [42]:
d._prev

{Value(data=2, grad=8.0), Value(data=4, grad=4.0)}

In [43]:
c = Value(3)
x = c ** 4 
x

Value(data=81, grad=0.0)

In [44]:
x.grad = 1
x._backward()
c._backward()
c

Value(data=3, grad=108.0)

In [45]:
x._prev

{Value(data=3, grad=108.0)}

In [46]:
a = Value(2)
b = Value(4)
c = Value(3)

d = a * b      # 8
f = d + c      # 11

f.backward()   # builds topo, sets f.grad=1, backprops

print(a, b, c) # grads: df/da = 4, df/db = 2, df/dc = 1

Value(data=2, grad=4.0) Value(data=4, grad=2.0) Value(data=3, grad=1.0)


In [47]:
# lets try a simple neuron

# inputs
x1 = Value(2)
x2 = Value(0)

# weights
w1 = Value(-3)
w2 = Value(1)

# biases
b2 = Value(6.8)

# wx+b
w1x1 = w1 * x1
w2x2 = w2 * x2
w1x1w2x2 = w1x1 + w2x2
n = w1x1w2x2 + b
o = n.tanh()

In [48]:
o.backward()

In [49]:
x = np.array((3,2), dtype=np.float32)

In [50]:
x[0]

np.float32(3.0)

In [51]:
mat = Value(np.random.randn(3,2))
scl = 3

res = mat * scl

In [ ]:
ix = Value([])
m2 = Value(np.random.rand(2, 3))

# initial prints
print(m1, m2)

res = m1@m2 # forward pass


In [65]:

xhat = () * Value(np.random.rand(2, 2)))
op = xhat + Value(0.1)

In [66]:
op

Value(data=[[0.51780413 0.10011145]
 [0.14046136 0.34343672]], grad=0.0)

In [67]:
op.backward()

In [68]:
op

Value(data=[[0.51780413 0.10011145]
 [0.14046136 0.34343672]], grad=1.0)